In [1]:
!pip install confluent-kafka

     |████████████████████████████████| 2.7 MB 30.5 MB/s            


In [17]:
!pip install simplejson

     |████████████████████████████████| 136 kB 40.7 MB/s            


In [10]:
consumer_key=""
consumer_secret=""
access_token_key=""
access_token_secret=""

In [42]:
from datetime import datetime

In [43]:
from TwitterAPI import TwitterAPI
api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret)

In [44]:
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka import Producer, Consumer


class Kafka:
    def __init__(self):
        self.adminClient = AdminClient({"bootstrap.servers": "localhost:9092"})

    def delivery_report(self, err, msg):
        try:
            if err is not None:
                print("Message delivery failed: {}".format(err))
            else:
                print(
                    "Message delivered to {} [{}]".format(
                        msg.topic(), msg.partition()
                    )
                )
        except Exception as err:
            print(err.__doc__)

    def create_topics(self, topic: str, num_partitions: int):
        new_topics = [
            NewTopic(topic=topic, num_partitions=num_partitions, replication_factor=1)
        ]
        fs = self.adminClient.create_topics(new_topics)
        for topic, f in fs.items():
            try:
                f.result()
                print("Topic {} created".format(topic))
            except Exception as e:
                print("Failed to create topic {}: {}".format(topic, e))

    def list_topics(self):
        print(self.adminClient.list_topics().topics)

    def produce_message(self, topic: str, key, value):
        producer = Producer({"bootstrap.servers": "localhost:9092"})
        producer.poll(0)
        try:
            producer.produce(
                topic=topic, key=key, value=value, callback=self.delivery_report
            )
            producer.flush()
        except Exception as err:
            print("Error:  {}".format(err.__doc__))

    def create_consumer(self, topic: str, group_id: str):
        consumer = Consumer(
            {
                "bootstrap.servers": "localhost:9092",
                "group.id": group_id,
                "auto.offset.reset": "earliest",
            }
        )

        consumer.subscribe([topic])
        return consumer

In [40]:
import simplejson as json
import datetime
r = api.request('statuses/filter', {'track':'bugün', 'language' : 'tr'})

class ProduceController:

    def __init__(self):
        self.fake = Faker()
        self.kafka = Kafka()

    def produce_tweets(self):
        r = api.request('statuses/filter', {'track':'pizza'})
        for item in r.get_iterator():
            try:
                json_object = json.dumps(item)
                tweet = json.loads(json_object)

                year = datetime.strptime(tweet['created_at'], "%a %b %d %H:%M:%S +0000 %Y").strftime('%Y-%m-%d').split('-')[0]
                month = datetime.strptime(tweet['created_at'], "%a %b %d %H:%M:%S +0000 %Y").strftime('%Y-%m-%d').split('-')[1]
                day = datetime.strptime(tweet['created_at'], "%a %b %d %H:%M:%S +0000 %Y").strftime('%Y-%m-%d').split('-')[2]
                credit_card = dict(
                            createdAt = int(year+month+day),
                          likeCount = tweet['favorite_count'],
                        name=tweet['user']['name'][0:3]+"***",
                         quoteCount = tweet['quote_count'],
                         replyCount = tweet['reply_count'],
                         retweetCount = tweet['retweet_count'],
                        text=tweet['text']) 
                print(credit_card)
            except (json.JSONDecodeError, KeyError) as err:
                print(
                    f"ERROR ===> 0of, encountered a mangled line of data here..\n"
                )
                continue

In [35]:
class AdminController:

    def create_topic(self, topic='tweet', num_partitions=2):
        try:
            kafka = Kafka()
            kafka.create_topics(topic=topic, num_partitions=num_partitions)
        except Exception as err:
            print(err.__doc__)

In [36]:
class CreditCardController:

    def create_credit_card_customer(self):
        kafka = Kafka()
        customer = kafka.create_consumer(
            topic='tweet',
            group_id='tweet')


        while True:
            try:
                msg = customer.poll()

                if msg is None:
                    continue
                if msg.error():
                    print("Consumer tweet error: {}".format(msg.error()))
                    continue
                
                json_data1 = json.loads(msg.value().decode('utf-8'))

                print(json_data1)
            except Exception as err:
                print(err.__doc__)

In [19]:
admin_controller = AdminController()
admin_controller.create_topic()

Failed to create topic credit_card: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'credit_card' already exists."}


In [ ]:
producer = ProduceController()

while True:
    producer.produce_credit_card_customer()

In [ ]:
credit_card_controller = CreditCardController()
credit_card_controller.create_credit_card_customer()